# Open Channel Flow Calculations

## This notebook shows how to calculate some common Open Channel Flow calculations such as:

1. $Q$ from $y_n$ (and channel properties) using either Manning's or Chezy equation
2. $y_n$ from $Q$ and other channel properties
3. Critical depth, $y_c$
4. Critical Slope, $So_c$
5. Solution of cubic equations e.g. for *Specific Energy* calculations 


## Chezy form of the Uniform Flow equation
$$
Q = ACR^{1/2}S_o^{1/2}
$$

## Manning's form of the Uniform Flow equation
$$
Q = \frac{1}{n} AR^{2/3}S_o^{1/2}
$$

## Chezy example $Q$ from $y_n$ for rectangular channel


Here are the equations for properties of a rectangular channel given the depth $y$
    
If flow in a $6m$ wide  rectangular channel is $4m$ deep running when in uniform flow what wiil be the discharge, $Q$ if the Chezy $C = 50 m^{1/2}/s$ and the slope of the channel is 1 in 1000?

We will need these equations of the channel properties:
![Properties of a rectangular channel](./images/channel_props_rectangular.png "Properties of a rectangular channel")



In [10]:
b = 6
C = 50
yn = 4
slope_v = 1
slope_h = 1000
So = slope_v/slope_h

A = b*yn
P = b + 2*yn
R = A/P

Q = A*C*(R**0.5)*(So**0.5)

print("Q = %6.3f m^3/s" % (Q))

Q = 49.685 m^3/s


## Manning's example $Q$ from $y_n$ for rectangular channel

Same example aas above but use Manning's $n = 0.02$
 



In [11]:
b = 6
n = 0.02
yn = 4
slope_v = 1
slope_h = 1000
So = slope_v/slope_h

A = b*yn
P = b + 2*yn
R = A/P

Q = A*(R**(2/3))*(So**0.5)/n

print("Q = %6.3f m^3/s" % (Q))

Q = 54.355 m^3/s


## Chezy equation calculate *normal depth*, $y_n$, from $Q$ and channel properties

For a $6m$ wide rectangular channel flowing in *uniform flow* with $Q = 30m^3/s$ calculate the normal depth $y_n$ 
if the Chezy $C = 50 m^{1/2}/s$ and the channel slopes with a drop of $1m$ in $1000m$

In [24]:

def secant_yn_rect_chezy(y1, y2, b, C, So, Q, debug):
    yn = (y1+y2)/2
    
    tol = 0.0005
    max_itts = 20
    
    for i in range(max_itts):

        A1 = b*y1
        P1 = b + 2*y1
        R1 = A1/P1
        Q1 = A1*C*(R1**(0.5))*(So**(0.5))
        f1 = Q-Q1
    
        A2 = b*y2
        P2 = b + 2*y2
        R2 = A2/P2
        Q2 = A2*C*(R2**(0.5))*(So**(0.5))
        f2 = Q-Q2

        dy = f2*(y2-y1)/(f2-f1)
        y_new = y2 - dy
        
        if(debug == 1):
            print("%d y1 = %6.3f m f1 = %6.3f m3/s   y2 = %6.3f m f2 = %6.3f m3/s dy = %6.4f" % (i,y1,f1,y2,f2,abs(dy)))
            

        if(abs(dy) < tol):
            y2 = y_new
            break
            
        y1 = y2
        y2 = y_new
    
    yn = y2
    return yn

Q = 30
b = 6
C = 50
slope_v = 1
slope_h = 1000
So = slope_v/slope_h

debug = 1

y1 = 1
y2 = 2
yn = secant_yn_rect_chezy(y1, y2, b, C, So, Q, debug)

print("yn = %6.3f m" % (yn))

0 y1 =  1.000 m f1 = 21.784 m3/s   y2 =  2.000 m f2 =  9.215 m3/s dy = 0.7332
1 y1 =  2.000 m f1 =  9.215 m3/s   y2 =  2.733 m f2 = -1.009 m3/s dy = 0.0724
2 y1 =  2.733 m f1 = -1.009 m3/s   y2 =  2.661 m f2 =  0.024 m3/s dy = 0.0017
3 y1 =  2.661 m f1 =  0.024 m3/s   y2 =  2.663 m f2 =  0.000 m3/s dy = 0.0000
yn =  2.663 m


In [40]:
import math
def secant_yn_trap_chezy(y1, y2, b, s, C, So, Q, debug):
    tol = 0.0005
    max_itts = 20

    for i in range(max_itts):

        A1 = (b + s*y1) * y1
        P1 = b + 2 * y1*math.sqrt(1+s*s)
        R1 = A1 / P1
        Q1 = A1 * C * (R1 ** (0.5)) * (So ** (0.5))
        f1 = Q - Q1
        if (debug == 2):
            print("%d A1 = %6.3f m2 P1 = %6.3f m   R1 = %6.3f m Q1 = %6.3f m3/s f1 = %6.4f" % (i, A1, P1, R1, Q1, f1))
        A2 = (b + s*y2) * y2
        P2 = b + 2 * y2*math.sqrt(1+s*s)
        R2 = A2 / P2
        Q2 = A2 * C * (R2 ** (0.5)) * (So ** (0.5))
        f2 = Q - Q2
        if (debug == 2):
            print("%d A2 = %6.3f m2 P2 = %6.3f m   R2 = %6.3f m Q2 = %6.3f m3/s f2 = %6.4f" % (i, A2, P2, R2, Q2, f2))

        dy = f2 * (y2 - y1) / (f2 - f1)
        y_new = y2 - dy

        if (debug > 0):
            print("%d y1 = %6.3f m f1 = %6.3f m3/s   y2 = %6.3f m f2 = %6.3f m3/s dy = %6.4f" % (i, y1, f1, y2, f2, abs(dy)))

        if (abs(dy) < tol):
            y2 = y_new
            break

        y1 = y2
        y2 = y_new

    yn = y2
    return yn


Q = 4.5
b = 4
s = 3
C = 80
slope_v = 1
slope_h = 100

So = slope_v / slope_h

debug = 0

y1 = 1
y2 = 2
yn = secant_yn_trap_chezy(y1, y2, b, s, C, So, Q, debug)

print("yn = %6.3f m" % (yn))


yn =  0.254 m


In [45]:
import math
def secant_yn_trap_manning(y1, y2, b, s, n, So, Q, debug):
    tol = 0.0005
    max_itts = 20

    for i in range(max_itts):

        A1 = (b + s*y1) * y1
        P1 = b + 2 * y1*math.sqrt(1+s*s)
        R1 = A1 / P1
        Q1 = A1 * (R1 ** (2/3)) * (So ** (0.5))/n
        f1 = Q - Q1
        if (debug == 2):
            print("%d A1 = %6.3f m2 P1 = %6.3f m   R1 = %6.3f m Q1 = %6.3f m3/s f1 = %6.4f" % (i, A1, P1, R1, Q1, f1))
        A2 = (b + s*y2) * y2
        P2 = b + 2 * y2*math.sqrt(1+s*s)
        R2 = A2 / P2
        Q2 = A2 *  (R2 ** (2/3)) * (So ** (0.5))/n
        f2 = Q - Q2
        if (debug == 2):
            print("%d A2 = %6.3f m2 P2 = %6.3f m   R2 = %6.3f m Q2 = %6.3f m3/s f2 = %6.4f" % (i, A2, P2, R2, Q2, f2))

        dy = f2 * (y2 - y1) / (f2 - f1)
        y_new = y2 - dy

        if (debug > 0):
            print("%d y1 = %6.3f m f1 = %6.3f m3/s   y2 = %6.3f m f2 = %6.3f m3/s dy = %6.4f" % (i, y1, f1, y2, f2, abs(dy)))

        if (abs(dy) < tol):
            y2 = y_new
            break

        y1 = y2
        y2 = y_new

    yn = y2
    return yn


Q = 30
b = 4
s = 2.5
n = 0.015
slope_v = 6.7
slope_h = 10000

So = slope_v / slope_h

debug = 1

y1 = 1
y2 = 2
yn = secant_yn_trap_manning(y1, y2, b, s, n, So, Q, debug)

print("yn = %6.3f m" % (yn))


0 y1 =  1.000 m f1 = 21.220 m3/s   y2 =  2.000 m f2 = -5.438 m3/s dy = 0.2040
1 y1 =  2.000 m f1 = -5.438 m3/s   y2 =  1.796 m f2 =  1.702 m3/s dy = 0.0486
2 y1 =  1.796 m f1 =  1.702 m3/s   y2 =  1.845 m f2 =  0.085 m3/s dy = 0.0026
3 y1 =  1.845 m f1 =  0.085 m3/s   y2 =  1.847 m f2 = -0.001 m3/s dy = 0.0000
yn =  1.847 m
